In [15]:
# Import libraries

import sys
import re
import requests
import asyncio
import nest_asyncio
import aiohttp
import time
import warnings
from datetime import datetime

# Suppress unnecessary Shapely warning
warnings.filterwarnings('ignore',
                        '.*Shapely GEOS version.*')

from aiohttp import ClientSession
from requests import request, Session
from itertools import product, repeat
import os
from dotenv import load_dotenv
from os import getenv
from threading import Thread
import time
import inspect
import pandas as pd
import geopandas as gp
import shapely
import pygeos
from functools import reduce
from pandas.plotting import lag_plot
import pickle
import numpy as np
import seaborn as sns
import datetime as dt
import copy
import math
from decimal import Decimal
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.ticker as mticker
from matplotlib.ticker import MaxNLocator
import matplotlib.dates as mdates
import matplotlib.gridspec as gridspec
from matplotlib.gridspec import GridSpec
from matplotlib.offsetbox import AnchoredText
import matplotlib as mpl
import plotly.express as px
from sklearn.metrics import mean_absolute_error
from multiprocess import Process, Pool

# Spark library
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.types import *
from pyspark.sql.functions import col as spark_col

# Using findspark ensures the spark path is
# found and set, allowing us to use the
# appropriate JAR files to connect to AWS
import findspark
location = findspark.find()
findspark.init(location, edit_rc=True)

# AWS library
import boto3
import awswrangler as wr


# All helper functions are in this module:
from helper_functions.census_functions import *

# Set up Pandas defaults
pd.options.display.float_format = '{:.4f}'.format
pd.set_option("display.max_columns", None)


# Connect to S3
s3client = boto3.client("s3")
os.environ.setdefault("AWS_PROFILE", "default")
os.environ.setdefault("AWS_DEFAULT_REGION", "us-west-1")

# Set environment variables for LOCAL development ONLY!
os.environ.setdefault("ENVIRON", "LOCAL")

'LOCAL'

## Get Census Key

In [2]:
# Test getting census key
print("Census key:", census_key)


Census key: f25144e3809a49b45c5cc54e7d2bce532fe4ce99


### Create the directories for file-saving

In [3]:
create_folder("datasets/cleaned_census_api_files/")
create_folder("datasets/cleaned_census_api_files/graphable/")
create_folder("datasets/cleaned_census_api_files/raw/")
create_folder("datasets/cleaned_census_api_files/standardized/")

## Following Tutorial Below
Referencing [this](https://www.youtube.com/watch?v=LW-M_UC0VTE) tutorial.

Here is the [Census API](https://www.census.gov/data/developers/data-sets.html)

## Get Block Group Level Datasets

In [3]:
# Set global variables
start_year = 2013
end_year = 2021

### Get Population by block group for 2013-2020

In [ ]:
%%time
# Start list
df_list = []
nest_asyncio.apply()

# Start session
session = Session()

# Define our API variable
# It's within a dictionary because some variables
# can change names from year to year (but not all)
census_code_dict = {
    2013: 'B01003_001E',
    2014: 'B01003_001E',
    2015: 'B01003_001E', # No change in code number this year
    2016: 'B01003_001E',
    2017: 'B01003_001E',
    2018: 'B01003_001E',
    2019: 'B01003_001E',
    2020: 'B01003_001E',
    2021: 'B01003_001E'
}

census_code_meaning='population_blocks'

# Run the API call
asyncio.run(url_to_dataframe_async_owners(start_year, end_year, 
                                          fifty_states_list=fifty_states_list,
                                          census_code_dict=census_code_dict,
                                          df_list=df_list,
                                          census_code_meaning=census_code_meaning,
                                         get_blocks=True))

# Get merged dataframe
pop_by_blocks_raw = final_data_prep(df_list, census_code_meaning, blocks=True)
pop_pre_st = merge_with_crosswalk(pop_by_blocks_raw)

# Drop name and block group
pop_pre_st.drop(columns=['name','block group'], inplace=True)

# Save crosswalked file
pop_pre_st.to_csv("datasets/cleaned_census_api_files/raw/merged_with_crosswalk/population_blocks_raw_crosswalked.csv",
                 index=False)

### Save pop_pre_st to S3

# Make block-group s3 path
block_group_s3_path = "s3://real-estate-wolff/census-data/block-groups/raw"
filename = "population_blocks_raw.csv"
s3_path = f"{block_group_s3_path}/{filename}"

# Save to s3
wr.s3.to_csv(pop_pre_st, s3_path, index=False)


In [48]:
# Make block-group s3 path
block_group_s3_path = "s3://real-estate-wolff/census-data/block-groups/raw"
filename = "population_blocks_raw.csv"
s3_path = f"{block_group_s3_path}/{filename}"

# Save to s3
wr.s3.to_csv(pop_pre_st, s3_path, index=False)

{'paths': ['s3://real-estate-wolff/census-data/block-groups/raw/population_blocks_raw.csv'],
 'partitions_values': {}}

In [25]:
# Read in the dataframe
pop_pre_st = pd.read_csv(
    "datasets/cleaned_census_api_files/raw/merged_with_crosswalk/population_blocks_raw_crosswalked.csv",
    encoding='utf-8',
    dtype={'geo_id':str, 'state':str, 'county':str, 
          'tract':str, 'block':str, 'BG10':str, 'BG20':str,
          'TRACT20':str, 'TRACT10':str})
pop_pre_st


,geo_id,2013,state,county,tract,block,2014,2015,2016,2017,2018,2019,2020,2021,BG10,BG20,wt_pop,wt_hu,wt_adult,wt_fam,wt_hh,parea,TRACT20,TRACT10
0,1500000US010010201001,637.0000,01,001,01001020100,010010201001,676.0000,649.0000,745.0000,692.0000,636.0000,730.0000,674.0000,693.0000,010010201001,010010201001,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,01001020100,01001020100
1,1500000US010010201001,637.0000,01,001,01001020100,010010201001,676.0000,649.0000,745.0000,692.0000,636.0000,730.0000,674.0000,693.0000,010010201001,010010201001,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,01001020100,01001020100
2,1500000US010010201002,1171.0000,01,001,01001020100,010010201002,1224.0000,1299.0000,1265.0000,1153.0000,1287.0000,1263.0000,1267.0000,1098.0000,010010201002,010010201002,1.0000,1.0000,1.0000,1.0000,1.0000,0.9988,01001020100,01001020100
3,1500000US010010201002,1171.0000,01,001,01001020100,010010201002,1224.0000,1299.0000,1265.0000,1153.0000,1287.0000,1263.0000,1267.0000,1098.0000,010010201002,010010208031,0.0000,0.0000,0.0000,0.0000,0.0000,0.0012,01001020803,01001020100
4,1500000US010010202001,1383.0000,01,001,01001020200,010010202001,1289.0000,1074.0000,960.0000,1020.0000,810.0000,835.0000,706.0000,844.0000,010010202001,010010202001,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,01001020200,01001020200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
561867,1500000US721537506012,3437.0000,72,153,72153750601,721537506012,2940.0000,2921.0000,2703.0000,2348.0000,2432.0000,2523.0000,1504.0000,1820.0000,721537506012,721537506013,0.6173,0.6167,0.6037,0.6202,0.6077,0.8305,72153750601,72153750601
561868,1500000US721537506013,1286.0000,72,153,72153750601,721537506013,1612.0000,1367.0000,1195.0000,1292.0000,976.0000,991.0000,1276.0000,1213.0000,721537506013,721537506011,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,72153750601,72153750601
561869,1500000US721537506021,2332.0000,72,153,72153750602,721537506021,2351.0000,1994.0000,2005.0000,2055.0000,1707.0000,1577.0000,1410.0000,1295.0000,721537506021,721537506021,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,72153750602,72153750602
561870,1500000US721537506022,856.0000,72,153,72153750602,721537506022,916.0000,747.0000,736.0000,946.0000,804.0000,648.0000,801.0000,894.0000,721537506022,721537506022,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,72153750602,72153750602


#### Standardize

In [ ]:
%%time
# This can and should be assigned to cloud parrallelization.
# The good thing is this function only needs to run once.
year_end = 2021
pop_dictionary = {}
array2 = pop_pre_st['BG20'].unique()

# Run the standardization function
[block_standardize(
    x, pop_dict=pop_dictionary, og_df=pop_pre_st, year_end=year_end) for x in array2]

saved_pop_dictionary = pop_dictionary.copy()

pop_standardized_df_1 = (pd.DataFrame.from_dict(saved_pop_dictionary, 
                       orient='index', 
                       columns=[str(i) for i in range(2013, year_end + 1)])
                       .reset_index()
                       .rename(columns={'index':'geoid_block'})
                      )

pop_standardized_df_1.to_csv('datasets/cleaned_census_api_files/raw/population_standardized_raw.csv',
                          encoding='utf-8',
                          index=False)

In [5]:
pop_pre_st.head(1)

,geo_id,name,2013,state,county,tract,block group,block,2014,2015,2016,2017,2018,2019,2020,2021,BG10,BG20,wt_pop,wt_hu,wt_adult,wt_fam,wt_hh,parea,TRACT20,TRACT10
0,1500000US010010201001,"Block Group 1, Census Tract 201, Autauga Count...",637.0000,01,001,01001020100,1,010010201001,676.0000,649.0000,745.0000,692.0000,636.0000,730.0000,674.0000,693.0000,010010201001,010010201001,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,01001020100,01001020100


In [10]:
pop_pre_st[
    pop_pre_st['BG20'].isin(
        pop_pre_st['BG20'].unique()[:20])]


,geo_id,name,2013,state,county,tract,block group,block,2014,2015,2016,2017,2018,2019,2020,2021,BG10,BG20,wt_pop,wt_hu,wt_adult,wt_fam,wt_hh,parea,TRACT20,TRACT10
0,1500000US010010201001,"Block Group 1, Census Tract 201, Autauga Count...",637.0000,01,001,01001020100,1,010010201001,676.0000,649.0000,745.0000,692.0000,636.0000,730.0000,674.0000,693.0000,010010201001,010010201001,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,01001020100,01001020100
1,1500000US010010201001,"Block Group 1, Census Tract 201, Autauga Count...",637.0000,01,001,01001020100,1,010010201001,676.0000,649.0000,745.0000,692.0000,636.0000,730.0000,674.0000,693.0000,010010201001,010010201001,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,01001020100,01001020100
2,1500000US010010201002,"Block Group 2, Census Tract 201, Autauga Count...",1171.0000,01,001,01001020100,2,010010201002,1224.0000,1299.0000,1265.0000,1153.0000,1287.0000,1263.0000,1267.0000,1098.0000,010010201002,010010201002,1.0000,1.0000,1.0000,1.0000,1.0000,0.9988,01001020100,01001020100
3,1500000US010010201002,"Block Group 2, Census Tract 201, Autauga Count...",1171.0000,01,001,01001020100,2,010010201002,1224.0000,1299.0000,1265.0000,1153.0000,1287.0000,1263.0000,1267.0000,1098.0000,010010201002,010010208031,0.0000,0.0000,0.0000,0.0000,0.0000,0.0012,01001020803,01001020100
4,1500000US010010202001,"Block Group 1, Census Tract 202, Autauga Count...",1383.0000,01,001,01001020200,1,010010202001,1289.0000,1074.0000,960.0000,1020.0000,810.0000,835.0000,706.0000,844.0000,010010202001,010010202001,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,01001020200,01001020200
5,1500000US010010202002,"Block Group 2, Census Tract 202, Autauga Count...",972.0000,01,001,01001020200,2,010010202002,1053.0000,1082.0000,1236.0000,1152.0000,1218.0000,1124.0000,1051.0000,1166.0000,010010202002,010010202002,1.0000,1.0000,1.0000,1.0000,1.0000,0.9962,01001020200,01001020200
6,1500000US010010202002,"Block Group 2, Census Tract 202, Autauga Count...",972.0000,01,001,01001020200,2,010010202002,1053.0000,1082.0000,1236.0000,1152.0000,1218.0000,1124.0000,1051.0000,1166.0000,010010202002,010010201001,0.0000,0.0000,0.0000,0.0000,0.0000,0.0038,01001020100,01001020200
7,1500000US010010203001,"Block Group 1, Census Tract 203, Autauga Count...",2366.0000,01,001,01001020300,1,010010203001,2376.0000,2143.0000,2364.0000,2555.0000,2641.0000,2774.0000,2912.0000,2685.0000,010010203001,010010203001,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,01001020300,01001020300
8,1500000US010010203002,"Block Group 2, Census Tract 203, Autauga Count...",691.0000,01,001,01001020300,2,010010203002,921.0000,825.0000,772.0000,830.0000,835.0000,733.0000,782.0000,892.0000,010010203002,010010203002,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,01001020300,01001020300
9,1500000US010010204001,"Block Group 1, Census Tract 204, Autauga Count...",1088.0000,01,001,01001020400,1,010010204001,970.0000,785.0000,729.0000,651.0000,653.0000,761.0000,720.0000,733.0000,010010204001,010010204001,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,01001020400,01001020400


In [6]:
def block_standardize_tuple(tuple):
    """
    WARNING: This function alone takes a few seconds to complete
    per block group, but to standard all 242,333 block groups
    can take many, many hours to run.
    It would be wise to run this function on any type of
    parrallel processing, such as using Dask, or a GPU,
    or parrallelized cloud computing, as there is no
    serialization (the block groups can be standardized
    in no particular order).
    
    This function standardizes all block group rows. It 
    should be called in a loop or vectorized if possible,
    such as the example below. (Note, the example below
    may not be the most efficient way to loop through
    or vectorize the block groups.)
    
    ```
    # Loop through all population block groups
    # and standardize them
    pop_dictionary = {}
    array2 = pop_pre_st['BG20'].unique()
    [block_standardize(
            x, 
            pop_dict=pop_dictionary, 
            og_df=pop_pre_st) 
        for x in array2]
    ```
    
    Parameters:
        tuple (tuple): A tuple containing the below.
            block (str): The block_group to group by.
            og_df (DataFrame): The dataframe we are 
                standardizing from.
            year_start (int): Which year to start from.
            year_end (int): Which year to end from.
            weight (str): Which weight to use (such as 
                'wt_pop' pr 'wt_hh').
    
    Returns:
        None. However, it appends the standardized values
            per block group to a pre-defined dictionary.
    """

    block_df = tuple[0]
    og_df = tuple[1]
    block = tuple[2]
    year_start = tuple[3]
    year_end = tuple[4]
    weight = tuple[5]
    
    years_10_19 = [str(i) for i in range(year_start, 2020)]
    years_2020_on = [str(i) for i in range(2020, end_year + 1)]
        
    # Step 1: Get a dataframe grouped by BG20
    df = og_df.copy()
    bg20_df = block_df.drop_duplicates()
    bg20_df = bg20_df.fillna(0)
        
    # Step 2: Get dot product of 2010-2019 values with the target weight values values
    array_10_19 = bg20_df[years_10_19].to_numpy().T
    wt_array = bg20_df[weight].to_numpy()
    dots = array_10_19.dot(wt_array)
    
    # Step 3: Append standardized 2010-2019 and 
    # the block's 2020 value to new dictionary
    filtered = df[df['block']==block]
    val_20_on = filtered[years_2020_on].iloc[0]
    dots = np.append(dots, val_20_on)

    # If the 2020 value is 0, then all years
    # before then should be 0 also
    if val_20_on[0] == 0:
        dots = dots * 0

    return {block : dots}
        

In [106]:
pop_pre_st[pop_pre_st['block'] == '010010206002']

,geo_id,name,2013,state,county,tract,block group,block,2014,2015,2016,2017,2018,2019,2020,2021,BG10,BG20,wt_pop,wt_hu,wt_adult,wt_fam,wt_hh,parea,TRACT20,TRACT10
29,1500000US010010206002,"Block Group 2, Census Tract 206, Autauga Count...",800.0000,01,001,01001020600,2,010010206002,832.0000,694.0000,712.0000,910.0000,1150.0000,1118.0000,644.0000,590.0000,010010206002,010010206001,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,01001020600,01001020600


In [7]:
stand_testing = pop_pre_st[
    pop_pre_st['BG20'].isin(
        pop_pre_st['BG20'].unique()[:20])].copy()


In [117]:
%%time

# testing numbers of blocks
stand_testing = pop_pre_st[
    pop_pre_st['BG20'].isin(
        pop_pre_st['BG20'].unique()[:400])].copy()


# Run the standardization using the block_standardize_tuple
begin_year = 2013
end_year = 2021

main_df = stand_testing.copy()

my_groups = main_df.groupby('BG20')
keys = list(my_groups.groups.keys())

dict2 = {}
dict3 = [block_standardize_tuple((my_groups.get_group(keys[i]),
                                pop_pre_st,
                                keys[i], 
                                begin_year, 
                                end_year, 
                                'wt_pop'))
        for i in range(len(keys))
       ]


for d in dict3:
    dict2.update(d)
    



CPU times: user 45.5 s, sys: 4.29 s, total: 49.7 s
Wall time: 2min 42s


In [118]:
# Time to run number of blocks without parallelization
{
    20:'4.31 s',
    200:'23.6 s',
    400:'162 s'
}

{20: '4.31 s', 200: '23.6 s', 400: '162 s'}

In [110]:
dict2

{'010010201001': array([637., 676., 649., 745., 692., 636., 730., 674., 693.]),
 '010010201002': array([1171., 1224., 1299., 1265., 1153., 1287., 1263., 1267., 1098.]),
 '010010202001': array([1383., 1289., 1074.,  960., 1020.,  810.,  835.,  706.,  844.]),
 '010010202002': array([ 972., 1053., 1082., 1236., 1152., 1218., 1124., 1051., 1166.]),
 '010010203001': array([2366., 2376., 2143., 2364., 2555., 2641., 2774., 2912., 2685.]),
 '010010203002': array([691., 921., 825., 772., 830., 835., 733., 782., 892.]),
 '010010204001': array([1088.,  970.,  785.,  729.,  651.,  653.,  761.,  720.,  733.]),
 '010010204002': array([1895., 1884., 2228., 2303., 2217., 1903., 1756., 1713., 1655.]),
 '010010204003': array([966., 949., 909., 978., 813., 705., 866., 707., 742.]),
 '010010204004': array([454., 469., 501., 553., 586., 570., 495., 399., 672.]),
 '010010205011': array([1859., 1961., 1936., 1947., 2068., 2150., 1972., 2006., 1847.]),
 '010010205012': array([1991., 1927., 1595., 1332., 1767.

In [111]:
column_list = []
for i in range(begin_year, end_year + 1):
    column_list.append(f'{i}')

standardized_df = (pd.DataFrame.from_dict(dict2, 
                   orient='index', columns=column_list)
                   .reset_index()
                   .rename(columns={'index':'geoid_block'}))

standardized_df

,geoid_block,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,010010201001,637.0000,676.0000,649.0000,745.0000,692.0000,636.0000,730.0000,674.0000,693.0000
1,010010201002,1171.0000,1224.0000,1299.0000,1265.0000,1153.0000,1287.0000,1263.0000,1267.0000,1098.0000
2,010010202001,1383.0000,1289.0000,1074.0000,960.0000,1020.0000,810.0000,835.0000,706.0000,844.0000
3,010010202002,972.0000,1053.0000,1082.0000,1236.0000,1152.0000,1218.0000,1124.0000,1051.0000,1166.0000
4,010010203001,2366.0000,2376.0000,2143.0000,2364.0000,2555.0000,2641.0000,2774.0000,2912.0000,2685.0000
5,010010203002,691.0000,921.0000,825.0000,772.0000,830.0000,835.0000,733.0000,782.0000,892.0000
6,010010204001,1088.0000,970.0000,785.0000,729.0000,651.0000,653.0000,761.0000,720.0000,733.0000
7,010010204002,1895.0000,1884.0000,2228.0000,2303.0000,2217.0000,1903.0000,1756.0000,1713.0000,1655.0000
8,010010204003,966.0000,949.0000,909.0000,978.0000,813.0000,705.0000,866.0000,707.0000,742.0000
9,010010204004,454.0000,469.0000,501.0000,553.0000,586.0000,570.0000,495.0000,399.0000,672.0000


In [6]:
### TEST ASSIGNING THIS TO SPARK
spark = SparkSession. \
    builder. \
    master('local'). \
    config("spark.driver.bindAddress","127.0.0.1"). \
    config("spark.driver.host","127.0.0.1"). \
    config("spark.jars.packages", 
           "org.apache.hadoop:hadoop-aws:2.10.2,org.apache.hadoop:hadoop-client:2.10.2"). \
    config("spark.jars.excludes", 
           "com.google.guava:guava"). \
    appName('Standardize Block Groups'). \
    getOrCreate()

### IF WORKING LOCALLY, GET AWS CREDENTIALS USING THE BELOW
if os.environ.get("ENVIRON") == "LOCAL":
    spark.sparkContext._jsc.hadoopConfiguration().set(
        "fs.s3a.aws.credentials.provider",
        "com.amazonaws.auth.DefaultAWSCredentialsProviderChain"
    )
    print("Set Default AWS Credentials Provider")

spark




23/02/15 17:09:31 WARN Utils: Your hostname, Austin-Wolffs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 172.16.225.161 instead (on interface en0)
23/02/15 17:09:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/Users/WonderWolff/Real_Estate/housing_supply_and_demand/venv/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/WonderWolff/.ivy2/cache
The jars for the packages stored in: /Users/WonderWolff/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
org.apache.hadoop#hadoop-client added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-7c162c05-8789-4ca1-9187-56ddb8e14167;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;2.10.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.271 in central
	found org.apache.commons#commons-lang3;3.4 in central
	found org.apache.hadoop#hadoop-client;2.10.2 in central
	found org.apache.hadoop#hadoop-common;2.10.2 in central
	found org.apache.hadoop#hadoop-annotations;2.10.2 in central
	found commons-cli#commons-cli;1.2 in central
	found org.apache.commons#commons-math3;3.1.1 in central
	found xmlenc#xmlenc;0.52 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found commons-logging#commons-logging

	[SUCCESSFUL ] org.apache.httpcomponents#httpclient;4.5.13!httpclient.jar (1594ms)
downloading https://repo1.maven.org/maven2/commons-codec/commons-codec/1.4/commons-codec-1.4.jar ...
	[SUCCESSFUL ] commons-codec#commons-codec;1.4!commons-codec.jar (205ms)
downloading https://repo1.maven.org/maven2/commons-io/commons-io/2.5/commons-io-2.5.jar ...
	[SUCCESSFUL ] commons-io#commons-io;2.5!commons-io.jar (436ms)
downloading https://repo1.maven.org/maven2/commons-net/commons-net/3.1/commons-net-3.1.jar ...
	[SUCCESSFUL ] commons-net#commons-net;3.1!commons-net.jar (453ms)
downloading https://repo1.maven.org/maven2/commons-collections/commons-collections/3.2.2/commons-collections-3.2.2.jar ...
	[SUCCESSFUL ] commons-collections#commons-collections;3.2.2!commons-collections.jar (925ms)
downloading https://repo1.maven.org/maven2/org/mortbay/jetty/jetty-sslengine/6.1.26/jetty-sslengine-6.1.26.jar ...
	[SUCCESSFUL ] org.mortbay.jetty#jetty-sslengine;6.1.26!jetty-sslengine.jar (94ms)
downloading

	[SUCCESSFUL ] log4j#log4j;1.2.17!log4j.jar(bundle) (780ms)
downloading https://repo1.maven.org/maven2/jline/jline/0.9.94/jline-0.9.94.jar ...
	[SUCCESSFUL ] jline#jline;0.9.94!jline.jar (131ms)
downloading https://repo1.maven.org/maven2/com/squareup/okhttp/okhttp/2.7.5/okhttp-2.7.5.jar ...
	[SUCCESSFUL ] com.squareup.okhttp#okhttp;2.7.5!okhttp.jar (526ms)
downloading https://repo1.maven.org/maven2/com/squareup/okio/okio/1.6.0/okio-1.6.0.jar ...
	[SUCCESSFUL ] com.squareup.okio#okio;1.6.0!okio.jar (101ms)
downloading https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-mapreduce-client-common/2.10.2/hadoop-mapreduce-client-common-2.10.2.jar ...
	[SUCCESSFUL ] org.apache.hadoop#hadoop-mapreduce-client-common;2.10.2!hadoop-mapreduce-client-common.jar (1260ms)
downloading https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-mapreduce-client-shuffle/2.10.2/hadoop-mapreduce-client-shuffle-2.10.2.jar ...
	[SUCCESSFUL ] org.apache.hadoop#hadoop-mapreduce-client-shuffle;2.10.2!hadoop-map

	91 artifacts copied, 0 already retrieved (129475kB/805ms)


23/02/15 17:13:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [33]:
spark.stop()

In [45]:
# # Set Hadoop configurations
# hadoop_conf = spark.sparkContext._jsc.hadoopConfiguration()
# hadoop_conf.set("fs.s3.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
# hadoop_conf.set("fs.s3a.endpoint", "s3.amazonaws.com")
# hadoop_conf.set("fs.s3a.aws.credentials.provider", 
#                 "org.apache.hadoop.fs.s3a.BasicAWSCredentialsProvider")
# hadoop_conf.set("fs.s3a.connection.ssl.enabled", "true")


In [55]:
# S3 URI
s3_location = "s3a://real-estate-wolff/census-data/block-groups/raw/population_blocks_raw.csv"

# READ IN DATAFRAME
spark_s3_df = spark.read.load(s3_location, format="csv", header=True)

# CHANGE TYPE FOR THE YEAR COLUMNS
for i in range(start_year, end_year + 1):
    spark_s3_df = spark_s3_df.withColumn(
        str(i), spark_col(str(i)).cast("float")
    )
    
# CHNAGE TYPE FOR THE CROSSWALK MULTIPLIER COLUMNS
spark_s3_df = spark_s3_df.withColumn('wt_pop', spark_col('wt_pop').cast("float")). \
    withColumn('wt_hu', spark_col('wt_hu').cast("float")). \
    withColumn('wt_adult', spark_col('wt_adult').cast("float")). \
    withColumn('wt_fam', spark_col('wt_fam').cast("float")). \
    withColumn('wt_hh', spark_col('wt_hh').cast("float")). \
    withColumn('parea', spark_col('parea').cast("float"))


spark_s3_df.show(5)


#2)
#### THEN GET TO TESTING THE SPARK JOB LOCALLY FOR A SPARK DATAFRAME
#### WITH ONLY 200 UNIQUE BG20'S

#3)
#### THEN TEST SPARK JOB 'YARN' ON EMR FOR 200 UNIQUES, AND TIME IT AND 
#### TRACK COST. THEN TRACK TIME AND COST FOR 2,000. IF NEED BE, ALSO
#### RUN THE EXPERIMENT WITH 22,000 UNIQUES, TRACKING TIME AND COST



+--------------------+------+-----+------+-----------+------------+------+------+------+------+------+------+------+------+------------+------------+------+-----+--------+------+-----+-----------+-----------+-----------+
|              geo_id|  2013|state|county|      tract|       block|  2014|  2015|  2016|  2017|  2018|  2019|  2020|  2021|        BG10|        BG20|wt_pop|wt_hu|wt_adult|wt_fam|wt_hh|      parea|    TRACT20|    TRACT10|
+--------------------+------+-----+------+-----------+------------+------+------+------+------+------+------+------+------+------------+------------+------+-----+--------+------+-----+-----------+-----------+-----------+
|1500000US01001020...| 637.0|   01|   001|01001020100|010010201001| 676.0| 649.0| 745.0| 692.0| 636.0| 730.0| 674.0| 693.0|010010201001|010010201001|   1.0|  1.0|     1.0|   1.0|  1.0|        1.0|01001020100|01001020100|
|1500000US01001020...| 637.0|   01|   001|01001020100|010010201001| 676.0| 649.0| 745.0| 692.0| 636.0| 730.0| 674.0|

In [112]:
spark_s3_df.printSchema()

root
 |-- geo_id: string (nullable = true)
 |-- 2013: float (nullable = true)
 |-- state: string (nullable = true)
 |-- county: string (nullable = true)
 |-- tract: string (nullable = true)
 |-- block: string (nullable = true)
 |-- 2014: float (nullable = true)
 |-- 2015: float (nullable = true)
 |-- 2016: float (nullable = true)
 |-- 2017: float (nullable = true)
 |-- 2018: float (nullable = true)
 |-- 2019: float (nullable = true)
 |-- 2020: float (nullable = true)
 |-- 2021: float (nullable = true)
 |-- BG10: string (nullable = true)
 |-- BG20: string (nullable = true)
 |-- wt_pop: float (nullable = true)
 |-- wt_hu: float (nullable = true)
 |-- wt_adult: float (nullable = true)
 |-- wt_fam: float (nullable = true)
 |-- wt_hh: float (nullable = true)
 |-- parea: float (nullable = true)
 |-- TRACT20: string (nullable = true)
 |-- TRACT10: string (nullable = true)



In [65]:
# Test getting 400 unique blocks
spark_key_list = [i['BG20'] for i in spark_s3_df.select('BG20').limit(10).collect()]


In [114]:
type(spark_s3_df)

pyspark.sql.dataframe.DataFrame

In [113]:
# CONTINUE TEST: only include blocks in our limited list
spark_testing = spark_s3_df.filter(spark_col('BG20').isin(spark_key_list))
spark_testing.show()

+--------------------+------+-----+------+-----------+------------+------+------+------+------+------+------+------+------+------------+------------+----------+----------+----------+----------+---------+------------+-----------+-----------+
|              geo_id|  2013|state|county|      tract|       block|  2014|  2015|  2016|  2017|  2018|  2019|  2020|  2021|        BG10|        BG20|    wt_pop|     wt_hu|  wt_adult|    wt_fam|    wt_hh|       parea|    TRACT20|    TRACT10|
+--------------------+------+-----+------+-----------+------------+------+------+------+------+------+------+------+------+------------+------------+----------+----------+----------+----------+---------+------------+-----------+-----------+
|1500000US01001020...| 637.0|   01|   001|01001020100|010010201001| 676.0| 649.0| 745.0| 692.0| 636.0| 730.0| 674.0| 693.0|010010201001|010010201001|       1.0|       1.0|       1.0|       1.0|      1.0|         1.0|01001020100|01001020100|
|1500000US01001020...| 637.0|   01| 

In [90]:
# CONTINUE TEST, GROUP BY BG20
spark_testing_groups = spark_testing.groupby("BG20")

In [17]:
# testing numbers of blocks
stand_testing = pop_pre_st[
    pop_pre_st['BG20'].isin(
        pop_pre_st['BG20'].unique()[:400])].copy()

# Drop unnecessary columns
stand_testing.drop(columns=['name','block group'], inplace=True)

spark_df = spark.createDataFrame(stand_testing)
spark_df

,geo_id,2013,state,county,tract,block,2014,2015,2016,2017,2018,2019,2020,2021,BG10,BG20,wt_pop,wt_hu,wt_adult,wt_fam,wt_hh,parea,TRACT20,TRACT10
0,1500000US010010201001,637.0000,01,001,01001020100,010010201001,676.0000,649.0000,745.0000,692.0000,636.0000,730.0000,674.0000,693.0000,010010201001,010010201001,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,01001020100,01001020100
1,1500000US010010201001,637.0000,01,001,01001020100,010010201001,676.0000,649.0000,745.0000,692.0000,636.0000,730.0000,674.0000,693.0000,010010201001,010010201001,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,01001020100,01001020100
2,1500000US010010201002,1171.0000,01,001,01001020100,010010201002,1224.0000,1299.0000,1265.0000,1153.0000,1287.0000,1263.0000,1267.0000,1098.0000,010010201002,010010201002,1.0000,1.0000,1.0000,1.0000,1.0000,0.9988,01001020100,01001020100
3,1500000US010010201002,1171.0000,01,001,01001020100,010010201002,1224.0000,1299.0000,1265.0000,1153.0000,1287.0000,1263.0000,1267.0000,1098.0000,010010201002,010010208031,0.0000,0.0000,0.0000,0.0000,0.0000,0.0012,01001020803,01001020100
4,1500000US010010202001,1383.0000,01,001,01001020200,010010202001,1289.0000,1074.0000,960.0000,1020.0000,810.0000,835.0000,706.0000,844.0000,010010202001,010010202001,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,01001020200,01001020200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,1500000US010179543005,1050.0000,01,017,01017954300,010179543005,1136.0000,1209.0000,1119.0000,1000.0000,664.0000,705.0000,NaN,NaN,010179543005,010179543003,1.0000,1.0000,1.0000,1.0000,1.0000,0.9995,01017954300,01017954300
944,1500000US010179543005,1050.0000,01,017,01017954300,010179543005,1136.0000,1209.0000,1119.0000,1000.0000,664.0000,705.0000,NaN,NaN,010179543005,010179548005,0.0000,0.0000,0.0000,0.0000,0.0000,0.0005,01017954800,01017954300
945,1500000US010179544001,817.0000,01,017,01017954400,010179544001,868.0000,906.0000,891.0000,850.0000,822.0000,796.0000,NaN,NaN,010179544001,010179548005,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,01017954800,01017954400
961,1500000US010179548005,NaN,01,017,01017954800,010179548005,NaN,NaN,NaN,NaN,NaN,NaN,722.0000,728.0000,010179543005,010179548005,0.0000,0.0000,0.0000,0.0000,0.0000,0.0005,01017954800,01017954300


In [19]:
spark_groups = spark_df.groupby("BG20")
spark_groups

In [105]:
og_df = wr.s3.read_csv(
    "s3://real-estate-wolff/census-data/block-groups/raw/population_blocks_raw.csv",
    encoding='utf-8',
    dtype={'geo_id':str, 'state':str, 'county':str, 
          'tract':str, 'block':str, 'BG10':str, 'BG20':str,
          'TRACT20':str, 'TRACT10':str}
)

og_df

,geo_id,2013,state,county,tract,block,2014,2015,2016,2017,2018,2019,2020,2021,BG10,BG20,wt_pop,wt_hu,wt_adult,wt_fam,wt_hh,parea,TRACT20,TRACT10
0,1500000US010010201001,637.0000,01,001,01001020100,010010201001,676.0000,649.0000,745.0000,692.0000,636.0000,730.0000,674.0000,693.0000,010010201001,010010201001,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,01001020100,01001020100
1,1500000US010010201001,637.0000,01,001,01001020100,010010201001,676.0000,649.0000,745.0000,692.0000,636.0000,730.0000,674.0000,693.0000,010010201001,010010201001,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,01001020100,01001020100
2,1500000US010010201002,1171.0000,01,001,01001020100,010010201002,1224.0000,1299.0000,1265.0000,1153.0000,1287.0000,1263.0000,1267.0000,1098.0000,010010201002,010010201002,1.0000,1.0000,1.0000,1.0000,1.0000,0.9988,01001020100,01001020100
3,1500000US010010201002,1171.0000,01,001,01001020100,010010201002,1224.0000,1299.0000,1265.0000,1153.0000,1287.0000,1263.0000,1267.0000,1098.0000,010010201002,010010208031,0.0000,0.0000,0.0000,0.0000,0.0000,0.0012,01001020803,01001020100
4,1500000US010010202001,1383.0000,01,001,01001020200,010010202001,1289.0000,1074.0000,960.0000,1020.0000,810.0000,835.0000,706.0000,844.0000,010010202001,010010202001,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,01001020200,01001020200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
561867,1500000US721537506012,3437.0000,72,153,72153750601,721537506012,2940.0000,2921.0000,2703.0000,2348.0000,2432.0000,2523.0000,1504.0000,1820.0000,721537506012,721537506013,0.6173,0.6167,0.6037,0.6202,0.6077,0.8305,72153750601,72153750601
561868,1500000US721537506013,1286.0000,72,153,72153750601,721537506013,1612.0000,1367.0000,1195.0000,1292.0000,976.0000,991.0000,1276.0000,1213.0000,721537506013,721537506011,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,72153750601,72153750601
561869,1500000US721537506021,2332.0000,72,153,72153750602,721537506021,2351.0000,1994.0000,2005.0000,2055.0000,1707.0000,1577.0000,1410.0000,1295.0000,721537506021,721537506021,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,72153750602,72153750602
561870,1500000US721537506022,856.0000,72,153,72153750602,721537506022,916.0000,747.0000,736.0000,946.0000,804.0000,648.0000,801.0000,894.0000,721537506022,721537506022,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,72153750602,72153750602


In [109]:
# Define global variable arguments
start_year = 2013
end_year = 2021
og_df = pop_pre_st.copy()
weight = "wt_pop"

def block_standardize_spark(key, pdf):

    # Get variables
    block = key[0]
    years_10_19 = [str(i) for i in range(start_year, 2020)]
    years_2020_on = [str(i) for i in range(2020, end_year + 1)]
    
    # Pandas Dataframe version
    df = og_df.copy()

    # Step 1: Get a dataframe grouped by BG20
    bg20_df = pdf.drop_duplicates().copy()
    bg20_df = bg20_df.fillna(0)

    # Step 2: Get dot product of 2010-2019 values with the target weight values values
    array_10_19 = bg20_df[years_10_19].to_numpy().T
    wt_array = bg20_df[weight].to_numpy()
    dots = array_10_19.dot(wt_array)

    # Step 3: Append standardized 2010-2019 and 
    # the block's 2020 value to new dictionary
    
    # This uses a Pandas Dataframe
    filtered = df[df['block']==block]
    val_20_on = filtered[years_2020_on].iloc[0]
    dots = np.append(dots, val_20_on)

    # If the 2020 value is 0, then all years
    # before then should be 0 also
    if val_20_on[0] == 0:
        dots = dots * 0
        
    # Convert to tuple
    tuple_dots = tuple(dots)

    return pd.DataFrame([key + tuple_dots])


In [ ]:
# Create  schema for the key and as many years as we have
schema2 = StructType([StructField('geoid_block', StringType(), True)] + 
                     [StructField(str(i), DoubleType(), True) 
                      for i in range(start_year, end_year + 1)])

# Run standardization function
spark_standardized = spark_groups.applyInPandas(
    block_standardize_spark, schema=schema2)

spark_standardized.show(5)

In [115]:
# TESTING CONTINUED
schema3 = StructType([StructField('geoid_block', StringType(), True)] + 
                     [StructField(str(i), DoubleType(), True) 
                      for i in range(start_year, end_year + 1)])

# Run standardization function
spark_test_standardized = spark_testing_groups.applyInPandas(
    block_standardize_spark, schema=schema3)

spark_test_standardized.show(5)


+------------+------+------+------+------+------+------+------+------+------+
| geoid_block|  2013|  2014|  2015|  2016|  2017|  2018|  2019|  2020|  2021|
+------------+------+------+------+------+------+------+------+------+------+
|010010201001| 637.0| 676.0| 649.0| 745.0| 692.0| 636.0| 730.0| 674.0| 693.0|
|010010201002|1171.0|1224.0|1299.0|1265.0|1153.0|1287.0|1263.0|1267.0|1098.0|
|010010202001|1383.0|1289.0|1074.0| 960.0|1020.0| 810.0| 835.0| 706.0| 844.0|
|010010202002| 972.0|1053.0|1082.0|1236.0|1152.0|1218.0|1124.0|1051.0|1166.0|
|010010203001|2366.0|2376.0|2143.0|2364.0|2555.0|2641.0|2774.0|2912.0|2685.0|
+------------+------+------+------+------+------+------+------+------+------+
only showing top 5 rows



In [77]:
spark_s3_df.filter(spark_s3_df['block']=='010010201001').show()

+--------------------+-----+-----+------+-----------+------------+-----+-----+-----+-----+-----+-----+-----+-----+------------+------------+------+-----+--------+------+-----+-----+-----------+-----------+
|              geo_id| 2013|state|county|      tract|       block| 2014| 2015| 2016| 2017| 2018| 2019| 2020| 2021|        BG10|        BG20|wt_pop|wt_hu|wt_adult|wt_fam|wt_hh|parea|    TRACT20|    TRACT10|
+--------------------+-----+-----+------+-----------+------------+-----+-----+-----+-----+-----+-----+-----+-----+------------+------------+------+-----+--------+------+-----+-----+-----------+-----------+
|1500000US01001020...|637.0|   01|   001|01001020100|010010201001|676.0|649.0|745.0|692.0|636.0|730.0|674.0|693.0|010010201001|010010201001|   1.0|  1.0|     1.0|   1.0|  1.0|  1.0|01001020100|01001020100|
|1500000US01001020...|637.0|   01|   001|01001020100|010010201001|676.0|649.0|745.0|692.0|636.0|730.0|674.0|693.0|010010201001|010010201001|   1.0|  1.0|     1.0|   1.0|  1.0| 

In [95]:
# Save spark dataframe to csv
spark_standardized.toPandas().to_csv(
    "datasets/cleaned_census_api_files/raw/population_spark_standardized_raw.csv",
    index=False
)

In [117]:
# Make block-group s3 path
block_group_s3_path = "s3://real-estate-wolff/census-data/block-groups/standardized"
filename = "population_blocks_standardized.csv"
s3_path = f"{block_group_s3_path}/{filename}"

# Save to s3
# wr.s3.to_csv(pop_pre_st.toPandas(), s3_path, index=False)
wr.s3.to_csv(spark_test_standardized.toPandas(), s3_path, index=False)

{'paths': ['s3://real-estate-wolff/census-data/block-groups/standardized/population_blocks_standardized.csv'],
 'partitions_values': {}}

In [165]:
spark.stop()

### Final dataframe clean up

In [ ]:
pop_standardized_df_1 = pd.read_csv('datasets/cleaned_census_api_files/raw/population_standardized_raw.csv',
                                  encoding='utf-8',
                                 dtype={'geoid_block':str})

# Run statistics function to get ratio and z-score
pop_standardized_df_1 = get_statistics(
    pop_standardized_df_1, begin_year=2013, end_year=2021)

# Add columns for state,county, and tract, and make sums for census tracts
pop_standardized_df_2 = specify_geographies(pop_standardized_df_1, 2013, 2021)
display(pop_standardized_df_2.head(1))

pop_standardized_df_2.to_csv('datasets/cleaned_census_api_files/standardized/population_standardized.csv',
                          encoding='utf-8',
                          index=False)

# Plot zscores to check for anomalies
plot_zscores(pop_standardized_df_2['z_score'], 'Population')

# Create the shapefiles for block groups and tracts
pop_tract_geo = create_and_save_geo_files(dataframe=pop_standardized_df_2, 
                                          name='population',
                                          begin_year=2013, 
                                          end_year=2021)



### Group Tracts By City

In [ ]:
##### Use the below function and incorporate it into the main
##### cencus helper functions. 

def group_tracts_by_city(tract_gdf, city_name, state_name):
    """
    Group tracts by a given city and save them as
    an individual dataframes.
    
    Parameters:
        tract_gdf (GeoDataFrame): The geodataframe
            you want to split by city.
        city_name (str): Name of city.
        state_name (str): Name of state.
    
    Returns:
        None.
    """
    
    # Read in the city shapefile
    city_boundaries = gp.read_file(
        "datasets/census_original_files/cities_2020/all_cities_2020/all_tracts_2020.shp")
    
    # Filter by city name and state
    
    # Make copy
    df = tract_gdf.copy()
    
    ### Below is an algorithm to only keep tracts
    ### that are within 50% or more of a city's boundary
    
    # Step 1: Calculate area of each tract
    df['tract_area'] = df.area
    
    # Step 2: Overlay the city boundaries over the tracts
    df_2 = df.overlay(city_boundaries, how="intersect")
    
    display(df_2)
    
    # Step 3: Calculate new areas
    df_2['area_']
    
group_tracts_by_city(pop_tract_geo)

In [ ]:
pop_tract_geo

In [ ]:
### Temporary code - can be deleted once file is created

# Goal: Create csv that matches states to FIPS codes



End of standardizing and merging population

---

## Median Gross Rent

Estimated Median of bin values = 𝑙 + (𝑛/2 − 𝐹)/𝑓 ⋅ 𝑤

#### Below is their "Median Value." Once we standardize, compare the values to this table below!

In [ ]:
# %%time
# # Start list
# df_list = []
# nest_asyncio.apply()

# # Start session
# session = Session()

# # Define our API variable
# # It's within a dictionary because some variables
# # can change names from year to year (but not all)
# census_code_dict = {
#     2013: 'B25064_001E',
#     2014: 'B25064_001E',
#     2015: 'B25064_001E', # No change in code number this year
#     2016: 'B25064_001E',
#     2017: 'B25064_001E',
#     2018: 'B25064_001E',
#     2019: 'B25064_001E',
#     2020: 'B25064_001E',
#     2021: 'B25064_001E'
# }

# census_code_meaning='median_rent_2013_2021_blocks'

# # Run the API call
# asyncio.run(url_to_dataframe_async_owners(2013, 2021, 
#                                           fifty_states_list,
#                                           census_code_dict,
#                                           df_list=df_list,
#                                           census_code_meaning=census_code_meaning,
#                                          get_blocks=True,
#                                          ))

# # Get merged dataframe
# median_rent_raw = final_data_prep(df_list, census_code_meaning, blocks=True)



In [ ]:
median_rent_raw = pd.read_csv('datasets/cleaned_census_api_files/raw/median_rent_2013_2021_blocks_raw.csv',
                                   encoding='utf-8',
                                   dtype={'geo_id':str, 'state':str, 'county':str, 
                                          'tract':str, 'block group':str, 'block':str})
median_rent_raw['block'] = median_rent_raw['geo_id'].apply(lambda x: str(x)[-12:])

spot_check(median_rent_raw, 2013, 2021)

median_rent_raw.head(1)


#### Get Gross Rent for each category, then combine them! Try to get all codes for all years

In [ ]:
# Get the Gross Rent Code for each year
rent_code_list = []
for i in range(3, 27):
    if i < 10:
        i = '0' + str(i)
    rent_code_list.append(f'B25063_0{i}E')
    
rent_code_list

In [ ]:
# %%time
# # Start list
# df_list = []
# nest_asyncio.apply()

# # Start session
# session = Session()

# # Define our API variable
# # It's within a dictionary because some variables
# # can change names from year to year (but not all)
# rent_code_list = []
# for i in range(3, 27):
#     if i < 10:
#         i = '0' + str(i)
#     rent_code_list.append(f'B25063_0{i}E')
    
# census_code_dict = {
#     2013: rent_code_list[:-3],
#     2014: rent_code_list[:-3],
#     2015: rent_code_list, 
#     2016: rent_code_list,
#     2017: rent_code_list,
#     2018: rent_code_list,
#     2019: rent_code_list,
#     2020: rent_code_list,
#     2021: rent_code_list
# }

# code_name_dict = {
#     'B25063_003E': 'rent_less_than_100',
#     'B25063_004E': 'rent_100_to_149',
#     'B25063_005E': 'rent_150_to_199',
#     'B25063_006E': 'rent_200_to_249',
#     'B25063_007E': 'rent_250_to_299',
#     'B25063_008E': 'rent_300_to_349',
#     'B25063_009E': 'rent_350_to_399',
#     'B25063_010E': 'rent_400_to_449',
#     'B25063_011E': 'rent_450_to_499',
#     'B25063_012E': 'rent_500_to_549',
#     'B25063_013E': 'rent_550_to_599',
#     'B25063_014E': 'rent_600_to_649',
#     'B25063_015E': 'rent_650_to_699',
#     'B25063_016E': 'rent_700_to_749',
#     'B25063_017E': 'rent_750_to_799',
#     'B25063_018E': 'rent_800_to_899',
#     'B25063_019E': 'rent_900_to_999',
#     'B25063_020E': 'rent_1000_to_1249',
#     'B25063_021E': 'rent_1250_to_1449',
#     'B25063_022E': 'rent_1500_to_1999',
#     'B25063_023E': 'rent_2000_to_2499',
#     'B25063_024E': 'rent_2500_to_2999',
#     'B25063_025E': 'rent_3000_to_3499',
#     'B25063_026E': 'rent_3500_or_more',
# }

# census_code_meaning='rent_distribution_blocks'

# # Run the API call
# asyncio.run(url_to_dataframe_async_owners(2013, 2021, 
#                                           fifty_states_list,
#                                           census_code_dict,
#                                           df_list=df_list,
#                                           census_code_meaning=census_code_meaning,
#                                          get_blocks=True,
#                                          multi_code=True,
#                                          code_name_dict=code_name_dict
#                                          ))

# # Get merged dataframe
# rent_distribution = final_data_prep(df_list, census_code_meaning, blocks=True)
# rent_dist_pre_st = merge_with_crosswalk(rent_dist)


In [ ]:
# # Custom spot checking
# print("How many states:", len(rent_dist['state'].value_counts()), "\n")

# for code in code_name_dict:
    
#     name = code_name_dict[code]
    
#     print(f"Values for {name}")
    
#     if (name != 'rent_2500_to_2999') and (name != 'rent_3000_to_3499') and (name != 'rent_3500_or_more'):
        
#         # Check for null values
#         for i in range(2013, 2021):
#             print(f"{name} Null values in {i}:", rent_dist[rent_dist[f'{i}_{name}'].isnull()].shape[0])
#         print("\n")

#         # Check for null values in multiple years
#         for i in range(2013, 2020):
#             print(f"2020 and {i} {name} null values:", rent_dist[(rent_dist[f'2020_{name}'].isnull()) & (rent_dist[f'{i}_{name}'].isnull())].shape[0])
#         print("\n")

#         # Check stats
#         for i in range(2013, 2021):
#             print(f"{name} Stats for year {i}:\n", rent_dist[f'{i}_{name}'].describe(), "\n")
            
#     else:
        
#         # Check for null values
#         for i in range(2015, 2021):
#             print(f"{name} Null values in {i}:", rent_dist[rent_dist[f'{i}_{name}'].isnull()].shape[0])
#         print("\n")

#         # Check for null values in multiple years
#         for i in range(2015, 2020):
#             print(f"2020 and {i} {name} null values:", rent_dist[(rent_dist[f'2020_{name}'].isnull()) & (rent_dist[f'{i}_{name}'].isnull())].shape[0])
#         print("\n")

#         # Check stats
#         for i in range(2015, 2021):
#             print(f"{name} Stats for year {i}:\n", rent_dist[f'{i}_{name}'].describe(), "\n")
        

In [ ]:
rent_dist = pd.read_csv('datasets/cleaned_census_api_files/raw/rent_distribution_blocks_raw.csv',
                        encoding='utf-8',
                       dtype={'geo_id':str, 'state':str, 'county':str, 
                                          'tract':str, 'block group':str, 'block':str})
rent_dist_pre_st = merge_with_crosswalk(rent_dist)
rent_dist_pre_st.head(1)

In [ ]:
def block_standardize_medians_3(bg20_df,
                            og_df,
                            year_start,
                            year_end,
                            weight,
                            code_name_dict_2013_2014=False,
                            code_name_dict_2015_2021=False,
                            code_name_dict_all=False):

    """
    WARNING: This function alone takes a few seconds to complete
    per block group, but to standard all 242,333 block groups
    can take many, many hours to run.
    It would be wise to run this function on any type of
    parrallel processing, such as using Dask, or a GPU,
    or parrallelized cloud computing, as there is no
    serialization (the block groups can be standardized
    in no particular order).
    
    This function standardizes all block group rows. It 
    should be called in a loop or vectorized if possible,
    such as the example below. (Note, the example below
    may not be the most efficient way to loop through
    or vectorize the block groups.)
    
    ```
    # Loop through all population block groups
    # and standardize them
    pop_dictionary = {}
    array2 = pop_pre_st['BG20'].unique()
    [block_standardize(
            x, 
            pop_dict=pop_dictionary, 
            og_df=pop_pre_st) 
        for x in array2]
    ```
    
    Parameters:
        tuple (tuple): A tuple containing the below.
            block (str): The block_group to group by.
            og_df (DataFrame): The dataframe we are 
                standardizing from.
            year_start (int): Which year to start from.
            year_end (int): Which year to end from.
            weight (str): Which weight to use (such as 
                'wt_pop' pr 'wt_hh').
    
    Returns:
        None. However, it appends the standardized values
            per block group to a pre-defined dictionary.
    """

    block = bg20_df['BG20'].iloc[0]
        
    # Step 1: Get a dataframe grouped by BG20
    bg20_df = bg20_df.drop_duplicates()
    bg20_df = bg20_df.fillna(0)
    filtered = og_df[og_df['block']==block].copy().drop_duplicates()
    
    # make sure the final dict is in the form of {block : dots}
    return_array = np.array([])
    
    if code_name_dict_all == False:
    
        # Step 2: Loop through the code names that 2013 and 2014 are guaranteed to have
        for code in code_name_dict_2013_2014:

            rent_category = code_name_dict_2013_2014[code]
            years_13_19 = [f"{i}_{rent_category}" for i in range(2013, 2020)]

            # Step 3: Get dot product of 2013-2019 values with the target weight values values
            array_13_19 = bg20_df[years_13_19].to_numpy().T
            wt_array = bg20_df[weight].to_numpy()
            dots = array_13_19.dot(wt_array)

            # Step 4: Append standardized 2013-2019 and 
            # the block's 2020+ values to new dictionary
            val_20s = filtered[[
                f"{i}_{rent_category}" for i in 
                range(2020, year_end + 1)]].iloc[0].to_numpy()
            
            # If the 2020 value is 0, then all years
            # before then should be 0 also
            if val_20s[0] == 0:
                dots = dots * 0

            # val_20 = filtered[f'2020_{rent_category}'].iloc[0]
            
            # Finalize the append
            dots = np.append(dots, val_20s)

            # Save a copy of the array
            set_dots = dots.copy()

            # Update return_dictionary
            return_array = np.append(return_array, set_dots)

        # Step 2: Loop through the code names that 2013 and 2014 won't have
        for code in code_name_dict_2015_2021:

            rent_category = code_name_dict_2015_2021[code]
            years_15_19 = [f"{i}_{rent_category}" for i in range(2015, 2020)]

            # Step 3: Get dot product of 2015-2019 values with the target weight values values
            array_15_19 = bg20_df[years_15_19].to_numpy().T
            wt_array = bg20_df[weight].to_numpy()
            dots = array_15_19.dot(wt_array)
            
            # Step 4: Append standardized 2013-2019 and 
            # the block's 2020+ values to new dictionary
            val_20s = filtered[[
                f"{i}_{rent_category}" for i in 
                range(2020, year_end + 1)]].iloc[0].to_numpy()
            
            # If the 2020 value is 0, then all years
            # before then should be 0 also
            if val_20s[0] == 0:
                dots = dots * 0
            
            # Finalize the append
            dots = np.append(dots, val_20s)

            # Save a copy of the array
            set_dots = dots.copy()

            # Update return_dictionary
            return_array = np.append(return_array, set_dots)
            
    else: # if code_name_dict_all exists
        for code in code_name_dict_all:

            rent_category = code_name_dict_all[code]
            years_13_19 = [f"{i}_{rent_category}" for i in range(2013, 2020)]

            # Step 3: Get dot product of 2010-2019 values with the target weight values values
            array_13_19 = bg20_df[years_13_19].to_numpy().T
            wt_array = bg20_df[weight].to_numpy()
            dots = array_13_19.dot(wt_array)
            
            # Step 4: Append standardized 2013-2019 and 
            # the block's 2020+ values to new dictionary
            val_20s = filtered[[
                f"{i}_{rent_category}" for i in 
                range(2020, year_end + 1)]].iloc[0].to_numpy()
            
            # If the 2020 value is 0, then all years
            # before then should be 0 also
            if val_20s[0] == 0:
                dots = dots * 0

            # Finalize the append
            dots = np.append(dots, val_20s)

            # Save a copy of the array
            set_dots = dots.copy()

            # Update return_dictionary
            return_array = np.append(return_array, set_dots)
        
    return {block : return_array}

In [ ]:
%%time
# Using list comprehension

begin_year = 2013
end_year = 2021

my_groups = rent_dist_pre_st.groupby('BG20')
keys = list(my_groups.groups.keys())

code_name_dict_2013_2014 = {
    'B25063_003E': 'rent_less_than_100',
    'B25063_004E': 'rent_100_to_149',
    'B25063_005E': 'rent_150_to_199',
    'B25063_006E': 'rent_200_to_249',
    'B25063_007E': 'rent_250_to_299',
    'B25063_008E': 'rent_300_to_349',
    'B25063_009E': 'rent_350_to_399',
    'B25063_010E': 'rent_400_to_449',
    'B25063_011E': 'rent_450_to_499',
    'B25063_012E': 'rent_500_to_549',
    'B25063_013E': 'rent_550_to_599',
    'B25063_014E': 'rent_600_to_649',
    'B25063_015E': 'rent_650_to_699',
    'B25063_016E': 'rent_700_to_749',
    'B25063_017E': 'rent_750_to_799',
    'B25063_018E': 'rent_800_to_899',
    'B25063_019E': 'rent_900_to_999',
    'B25063_020E': 'rent_1000_to_1249',
    'B25063_021E': 'rent_1250_to_1449',
    'B25063_022E': 'rent_1500_to_1999',
    'B25063_023E': 'rent_2000_to_2499'
}

code_name_dict_2015_2021 = {
    'B25063_024E': 'rent_2500_to_2999',
    'B25063_025E': 'rent_3000_to_3499',
    'B25063_026E': 'rent_3500_or_more'
}

dict2 = {}
dict3 = [block_standardize_medians_3(my_groups.get_group(keys[i]), 
                                rent_dist_pre_st, 
                                begin_year, 
                                end_year, 
                                'wt_hh', 
                                code_name_dict_2013_2014=code_name_dict_2013_2014, 
                                code_name_dict_2015_2021=code_name_dict_2015_2021)
        for i in range(len(keys))
       ]


for d in dict3:
    dict2.update(d)

column_list = []
for code in code_name_dict_2013_2014:
    for i in range(begin_year, end_year + 1):
        column_list.append(f'{i}_{code_name_dict_2013_2014[code]}')
for code in code_name_dict_2015_2021:
    for i in range(begin_year, end_year + 1):
        column_list.append(f'{i}_{code_name_dict_2015_2021[code]}')

gross_rent_df = (pd.DataFrame.from_dict(dict2, 
                   orient='index', columns=column_list)
                   .reset_index()
                   .rename(columns={'index':'geoid_block'}))

gross_rent_df.to_csv('datasets/cleaned_census_api_files/raw/gross_rent_standardized_raw.csv',
                          encoding='utf-8',
                          index=False)

gross_rent_df

In [ ]:
gross_rent_df.to_csv('datasets/cleaned_census_api_files/raw/gross_rent_standardized_raw.csv',
                          encoding='utf-8',
                          index=False)


In [ ]:
gross_rent_df = pd.read_csv('datasets/cleaned_census_api_files/raw/gross_rent_standardized_raw.csv',
                          encoding='utf-8',
                          dtype={'geoid_block':str})

# Make geoid_block the index
gross_rent_df.set_index('geoid_block', inplace=True)
gross_rent_df.head(2)


### Once I have the gross rents standardized, calculate the median value for each year using frequency tables

Estimated Median of bin values = 𝑙 + (𝑛/2 − 𝐹)/𝑓 ⋅ 𝑤

where 𝑙 is the lower border of the median group, 𝐹 is the cumulative frequency up to the median group, 𝑓 is the frequency of the median group, 𝑤 is the width of the median group.


In [ ]:
### STEPS TO ESTIMATE MEDIAN VALUE
def estimate_median(series, year, keyword):
    """
    Estimate the median for a given year.
    """
    s = series.copy()
    year = str(year)
#     print(series.name)
    
    # Gather all variables for the equation
    non_zero_series = s[s != 0]
    n = non_zero_series.sum()
    
    if len(non_zero_series) == 0:
        return 0
    
    # Find median interval
    median_n = n/2

    series_dict = non_zero_series.to_dict()
    key_list = list(series_dict.keys())
    count = 0
    interval = ''
    interval_backup = None
    i = 0
    while i < len(key_list):
        key = key_list[i]
        if count < median_n:
            interval = key
            count += series_dict[key]
            if count > median_n:
                F = count - series_dict[key]
        elif count == median_n:
            F = count
            interval_backup = key_list[i]
            i = len(key_list)
        else:
            pass
        i += 1
        
    f = series_dict[interval]
        
    # Get l, the lower bound
    interval_bound = interval.replace(f'{year}_{keyword}_','')
    l = int(re.sub(r'_to_\d+|less_than_|_or_more','',interval_bound))
    
    # Get higher bound
    h = int(re.sub(r'\d+_to_|less_than_|_or_more','',interval_bound))

    # Get width
    w = h - l
    
    # Calculate almost_median
    almost_median = (((n/2) - F)/f)*w
    
    # Check if there is no "median interval"
    if (almost_median == 0) & (interval_backup is not None):
        
        # Get l, the lower bound
        interval_higher_bound = interval_backup.replace(f'{year}_{keyword}_','')
        l_higher = int(re.sub(r'_to_\d+|less_than_|_or_more','',
                              interval_higher_bound))

        # Get higher bound
        h_higher = int(re.sub(r'\d+_to_|less_than_|_or_more','',interval_higher_bound))

        # Calculate average of lower-lower bound and higher-higher bound
        split_intervals_median = (l + h_higher)/2
        
        # This is our median
        return split_intervals_median
    
    else:
    
        # estimate median
        estimated_median = l + almost_median

        return estimated_median



In [ ]:
gross_rent_medians = gross_rent_df.copy()

In [ ]:
%%time
# Calculate gross rent for every year
for i in range(2013, 2021):
    columns_year = gross_rent_df.columns[gross_rent_df.columns.str.contains(str(i))]
    gross_rent_medians[f'{i}_median'] = gross_rent_df[columns_year].apply(
        lambda x: estimate_median(x, str(i), 'rent'),
        axis=1
    )
gross_rent_medians

In [ ]:
gross_rent_medians.to_csv('datasets/cleaned_census_api_files/raw/gross_rent_medians_raw.csv',
                          encoding='utf-8')

In [ ]:
gross_rent_medians = pd.read_csv('datasets/cleaned_census_api_files/raw/gross_rent_medians_raw.csv',
                          encoding='utf-8',
                          dtype={'geoid_block':str}
                                )
gross_rent_medians.head(2)

In [ ]:
gross_rent_medians = gross_rent_medians[['geoid_block','2013_median','2014_median','2015_median',
                                        '2016_median','2017_median','2018_median',
                                        '2019_median','2020_median']]
gross_rent_medians.head(2)

In [ ]:
# Create loop that goes through each year backyards, 
# and linearly interpolate values between 0 values 
# (to show one consistent line on a grpah between two points spearated by "0")
def linearly_interpolate(dataframe, 
                         grouping='block'):
    """Linearly interpolate NaN values."""
    df = copy.deepcopy(dataframe)
    df = df.replace(0, np.nan)
    df = df.set_index(f'geoid_{grouping}')
    df = df.interpolate(axis=1).reset_index()
    
    return df


In [ ]:
median_rent_interpolated = linearly_interpolate(gross_rent_medians)
median_rent_interpolated

In [ ]:
%%time
### Now get the median tract values
def get_tract_median_values(dataframe):
    "Sum blocks for tracts then calculate each year's median."
    
    df = copy.deepcopy(dataframe)
    if 'geoid_block' not in df.columns:
        df = df.reset_index()

    df['geoid_tract'] = df['geoid_block'].apply(lambda x: str(x)[:-1])
    df['state'] = df['geoid_block'].apply(lambda x: str(x)[0:2])
    df['county'] = df['geoid_block'].apply(lambda x: str(x)[2:5])
    
    # Calculate gross rent for every year
    for i in range(2013, 2021):
        str_year = str(i)
        columns_year = df.columns[df.columns.str.contains(str_year)]
        for col in columns_year:
            main_name = col.replace(f'{i}_rent_','')
            df[f'{i}_tract_rent_{main_name}'] = df.groupby('geoid_tract')[f'{col}'].transform('sum')
        tracts_year = df.columns[df.columns.str.contains(f'{str_year}_tract')]
        df[f'{i}_tract_median'] = df[tracts_year].apply(
            lambda x: estimate_median(x, str(i), 'tract_rent'),
            axis=1
        )
    
    return df

gross_rent_tract = get_tract_median_values(gross_rent_df)
gross_rent_tract

In [ ]:
gross_rent_tract_2 = gross_rent_tract[['geoid_tract',
                                       '2013_tract_median','2014_tract_median','2015_tract_median',
                                        '2016_tract_median','2017_tract_median','2018_tract_median',
                                        '2019_tract_median','2020_tract_median']].drop_duplicates()
gross_rent_tract_2

In [ ]:
median_tract_rent_interpolated = linearly_interpolate(gross_rent_tract_2, grouping='tract')
median_tract_rent_interpolated

In [ ]:
median_rent_interpolated['geoid_tract'] = median_rent_interpolated['geoid_block'].apply(
    lambda x: str(x)[:-1])

median_rent_interpolated

In [ ]:
median_rent_all = pd.merge(median_rent_interpolated,
                           median_tract_rent_interpolated,
                           left_on="geoid_tract",
                           right_on="geoid_tract",
                           how='inner')

median_rent_all

In [ ]:
for year in range(2013, 2021):
    median_rent_all.rename(columns={f'{year}_median': f'{year}_block_median'}, inplace=True)
    
median_rent_all


In [ ]:
median_rent_all['state'] = median_rent_all['geoid_block'].apply(lambda x: str(x)[0:2])
median_rent_all['county'] = median_rent_all['geoid_block'].apply(lambda x: str(x)[2:5])
median_rent_all['tract'] = median_rent_all['geoid_block'].apply(lambda x: str(x)[5:11])

median_rent_all

In [ ]:
median_rent_all.to_csv('datasets/cleaned_census_api_files/standardized/median_rent_standardized.csv',
                       encoding='utf-8',
                       index=False)

In [ ]:
median_rent_all = pd.read_csv('datasets/cleaned_census_api_files/standardized/median_rent_standardized.csv',
                               encoding='utf-8',
                             dtype={'geoid_block':str, 'geoid_tract':str})

median_rent_all


In [ ]:
### Once I have the interpolated median tract values, merge with block level,
# Save, then run create_and_save_geo_files() function on it

# Create the shapefiles for block groups and tracts
median_rent_block_geo, median_rent_tract_geo = create_and_save_geo_files(dataframe=median_rent_all, 
                                                          name='median_rent',
                                                          keyword='median',
                                                          begin_year=2013, 
                                                          end_year=2020)

In [ ]:
median_rent_block_geo
median_rent_tract_geo

### End of getting data at the Tract level
---

# Get data at the MSA level

We will get the following:
1. Median Income
2. Median Unit Price
3. Median Rent

using the ACS 1-Year survey.

In [ ]:
# Set end year for all MSAs
end_year=2021

### Get Median Income at the MSA level

In [ ]:
# Run the API download function
median_income_msa = download_and_format_msa_census_data(
    census_code="B19013_001E",
    census_code_meaning="median_income_msa",
    end_year=end_year
)
median_income_msa

### Get Median Unit Value at the MSA Level

In [ ]:
# Run the API download function
median_price_msa = download_and_format_msa_census_data(
    census_code="B25077_001E",
    census_code_meaning="median_price_msa",
    end_year=end_year
)
median_price_msa

### Get Median Rent at the MSA level

In [ ]:
# Run the API download function
median_rent_msa = download_and_format_msa_census_data(
    census_code="B25058_001E",
    census_code_meaning="median_rent_msa",
    end_year=end_year
)
median_rent_msa

### Get Total Units at the MSA level

In [ ]:
# Run the API download function
total_units_msa = download_and_format_msa_census_data(
    census_code="B25001_001E",
    census_code_meaning="total_units_msa",
    end_year=end_year
)
total_units_msa

### Create Rent-to-Price Ratio dataset

In [ ]:
# Rename columns
for i in range(2010, end_year + 1):
    median_rent_msa.rename(columns={f"{i}":f"{i}_rent"}, inplace=True)

# Rename columns
for i in range(2010, end_year + 1):
    median_price_msa.rename(columns={f"{i}":f"{i}_price"}, inplace=True)

# Merge price data
rent_to_price = median_rent_msa.merge(
    median_price_msa, how='inner', 
    on=['msa_code','msa_name'])

# Loop through columns and divide rent by price per year
for i in range(2010, end_year + 1):
    rent_to_price[f'{i}'] = rent_to_price[f"{i}_rent"]/rent_to_price[f"{i}_price"]
    
    # Drop rent and price columns
    rent_to_price.drop(columns=[f'{i}_rent',f'{i}_price'], inplace=True)

# Save dataset
rent_to_price.to_csv(
    "datasets/cleaned_census_api_files/msa_data/rent_price_ratio_msa.csv", 
    index=False)

rent_to_price

### Create Jobs per Unit dataset

In [ ]:
# Read in jobs
jobs = pd.read_csv('datasets/bls/raw/most_recent_bls_data.csv',
                   dtype={'msa_code':str, 'state_code':str})

# Make sure the date column is in datetime format
jobs['date'] = pd.to_datetime(jobs['date'])

# Replace NECTA Division
jobs['msa_name'] = jobs['msa_name'].apply(lambda x: x.replace(" NECTA Division",""))
jobs['msa_name'] = jobs['msa_name'].apply(lambda x: x.replace(" NECTA",""))

# Keep only december months
new_jobs = jobs[jobs['month']=='December'].reset_index(drop=True)

# Get earliest year
earliest_year = new_jobs['year'].min()

# Get latest year
latest_year = new_jobs['year'].max()

# Only keep certain columns
new_jobs = new_jobs[['msa_name','year','value']]

# Rename column
new_jobs.rename(columns={'value':f'jobs'}, inplace=True)

# Stack and unstack
new_jobs = new_jobs.set_index(['msa_name','year'])
new_jobs = new_jobs.unstack('year')

# Reset index
new_jobs = new_jobs.reset_index()

# Rename jobs columns
new_jobs.columns = ['msa_name'] + [
    f'{i}_jobs' for i in range(earliest_year, latest_year + 1)]

# Read in total units and rename columns
total_units = pd.read_csv(
    "datasets/cleaned_census_api_files/msa_data/total_units_msa.csv")
for i in range(earliest_year, latest_year + 1):
    total_units.rename(columns={f"{i}":f"{i}_units"}, inplace=True)
    
# Merge data
jobs_per_unit = new_jobs.merge(
    total_units, how='inner', 
    on=['msa_name'])

# Loop through columns and divide rent by price per year
for i in range(earliest_year, latest_year + 1):
    jobs_per_unit[f'{i}'] = jobs_per_unit[f"{i}_jobs"]/jobs_per_unit[f"{i}_units"]
        
# Only keep main columns
jobs_per_unit = jobs_per_unit[['msa_name','msa_code'] +
    [f'{i}' for i in range(earliest_year, latest_year + 1)]]

# Save dataset
jobs_per_unit.to_csv(
    "datasets/cleaned_census_api_files/msa_data/jobs_per_unit_msa.csv", 
    index=False)

jobs_per_unit



## Get data at the City Level (at the ACS 5-Year level)

1. Population (B01003_001E)
2. Median Income
3. Median Unit Price
4. Median Rent
5. Total Units
6. Percent Renter Occupied
7. Total Employed (B23025_004E)

Create Manually:
1. Rent-to-Price Ratio
2. People-per-Units

In [ ]:
# Define end year
begin_year = 2011
end_year = 2021

### Get Population (City)

In [ ]:
# Run the API download function
population_city = download_and_format_msa_census_data(
    census_code="B01001_001E",
    census_code_meaning="population_city",
    begin_year=begin_year,
    end_year=end_year,
    format_msa=False,
    format_city=True
)
population_city

### Get Median Income (city)

In [ ]:
# Run the API download function
median_income_city = download_and_format_msa_census_data(
    census_code="B19013_001E",
    census_code_meaning="median_income_city",
    begin_year=begin_year,
    end_year=end_year,
    format_msa=False,
    format_city=True
)
median_income_city

### Get Median Price (city)

In [ ]:
# Run the API download function
median_price_city = download_and_format_msa_census_data(
    census_code="B25077_001E",
    census_code_meaning="median_price_city",
    begin_year=begin_year,
    end_year=end_year,
    format_msa=False,
    format_city=True
)
median_price_city

### Get Median Rent (city)

In [ ]:
# Run the API download function
median_rent_city = download_and_format_msa_census_data(
    census_code="B25058_001E",
    census_code_meaning="median_rent_city",
    begin_year=begin_year,
    end_year=end_year,
    format_msa=False,
    format_city=True
)
median_rent_city

### Get Total Units (city)

In [ ]:
# Run the API download function
total_units_city = download_and_format_msa_census_data(
    census_code="B25001_001E",
    census_code_meaning="total_units_city",
    begin_year=begin_year,
    end_year=end_year,
    format_msa=False,
    format_city=True
)
total_units_city

## Get Percent Renter Occupied (city)

1. First we must get Total Occupied Units (B25002_002E)
2. Then we must get Renter Occupied Units (B25003_003E)

3. Then we must manually divide Renters by Occupied units to get Percent Renter Occupied

In [ ]:
# Run the API download function
total_occupied_city = download_and_format_msa_census_data(
    census_code="B25002_002E",
    census_code_meaning="total_occupied_city",
    begin_year=begin_year,
    end_year=end_year,
    format_msa=False,
    format_city=True
)
total_occupied_city

total_renter_occupied_city = download_and_format_msa_census_data(
    census_code="B25003_003E",
    census_code_meaning="total_renter_occupied_city",
    begin_year=begin_year,
    end_year=end_year,
    format_msa=False,
    format_city=True
)
total_renter_occupied_city

In [ ]:
### Manually get Percent Renter Occupied

# Rename columns
for i in range(begin_year, end_year + 1):
    total_occupied_city.rename(columns={f"{i}":f"{i}_total_occupied"}, inplace=True)

# Rename columns
for i in range(begin_year, end_year + 1):
    total_renter_occupied_city.rename(columns={f"{i}":f"{i}_renter_occupied"}, inplace=True)

# Merge price data
percent_renter_city = total_occupied_city.merge(
    total_renter_occupied_city, how='inner', 
    on=['name','geo_id'])

# Loop through columns and divide rent by price per year
for i in range(begin_year, end_year + 1):
    percent_renter_city[f'{i}'] = percent_renter_city[f"{i}_renter_occupied"]/percent_renter_city[f"{i}_total_occupied"]
    
    # Drop rent and price columns
    percent_renter_city.drop(columns=[
        f'{i}_renter_occupied',f'{i}_total_occupied'], inplace=True)

# Save dataset
percent_renter_city.to_csv(
    "datasets/cleaned_census_api_files/city_data/percent_renter_occupied_city.csv", 
    index=False)

percent_renter_city


### Get Total Employed (city)

In [ ]:
# Run the API download function
total_employed_city = download_and_format_msa_census_data(
    census_code="B23025_004E",
    census_code_meaning="total_employed_city",
    begin_year=begin_year,
    end_year=end_year,
    format_msa=False,
    format_city=True
)
total_employed_city

### Manually Create Rent-to-Price Ratio (city)


In [ ]:
### Manually get Rent to Price Ratio

# Rename columns
for i in range(begin_year, end_year + 1):
    median_rent_city.rename(columns={f"{i}":f"{i}_rent"}, inplace=True)

# Rename columns
for i in range(begin_year, end_year + 1):
    median_price_city.rename(columns={f"{i}":f"{i}_price"}, inplace=True)

# Merge price data
rent_price_ratio_city = median_rent_city.merge(
    median_price_city, how='inner', 
    on=['name','geo_id'])

# Loop through columns and divide rent by price per year
for i in range(begin_year, end_year + 1):
    rent_price_ratio_city[f'{i}'] = rent_price_ratio_city[f"{i}_rent"]/rent_price_ratio_city[f"{i}_price"]
    
    # Drop rent and price columns
    rent_price_ratio_city.drop(columns=[
        f'{i}_rent',f'{i}_price'], inplace=True)

# Save dataset
rent_price_ratio_city.to_csv(
    "datasets/cleaned_census_api_files/city_data/rent_price_ratio_city.csv", 
    index=False)

rent_price_ratio_city


### Manually Create People-per-Units (city)


In [ ]:
### Manually get People per Units

# Rename columns
for i in range(begin_year, end_year + 1):
    population_city.rename(columns={f"{i}":f"{i}_population"}, inplace=True)

# Rename columns
for i in range(begin_year, end_year + 1):
    total_units_city.rename(columns={f"{i}":f"{i}_units"}, inplace=True)

# Merge price data
people_per_unit_city = population_city.merge(
    total_units_city, how='inner', 
    on=['name','geo_id'])

# Loop through columns and divide rent by price per year
for i in range(begin_year, end_year + 1):
    people_per_unit_city[f'{i}'] = people_per_unit_city[f"{i}_population"]/people_per_unit_city[f"{i}_units"]
    
    # Drop rent and price columns
    people_per_unit_city.drop(columns=[
        f'{i}_population',f'{i}_units'], inplace=True)

# Save dataset
people_per_unit_city.to_csv(
    "datasets/cleaned_census_api_files/city_data/people_per_unit_city.csv", 
    index=False)

people_per_unit_city
